In [ ]:
import pandas as pd
import requests #Pushshift accesses Reddit via an url so this is needed
import json #JSON manipulation
import csv #To Convert final table into a csv file to save to your machine
import time
import datetime
from tqdm import tqdm
import networkx as nx

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!cp -r  /content/gdrive/MyDrive/gaming.csv '/content/'

Mounted at /content/gdrive


In [ ]:
G=nx.read_gexf('/content/gdrive/MyDrive/gaming_v5.gexf')
G_nodes = G.number_of_nodes()
G_edges = G.number_of_edges()
print("Nodes = ", G_nodes, " Edges = ",G_edges)

Nodes =  212842  Edges =  373026


In [ ]:
"""
Calculating page rank for all the nodes in the network.
"""
pageRank = nx.pagerank(G, alpha=0.9)
#print(pageRank)
f = open("pageRank.txt", "a")
for i in pageRank:
  x=str(i)+"-->"+str(pageRank[str(i)])
  f.write(str(x))
f.close()

In [ ]:
"""
Calculating degree centrality for all the nodes in the network.
"""
degreeCentrality = nx.degree_centrality(G)
print(len(degreeCentrality))
f = open("degreeCentrality.txt", "a")
for i in degreeCentrality:
  x=str(i)+"-->"+str(degreeCentrality[str(i)])
  f.write(str(x))
f.close()

212842


In [ ]:
"""
Downloading required package for sentiment analysis.
"""
import nltk
nltk.download(["names","stopwords","state_union","twitter_samples","movie_reviews","averaged_perceptron_tagger","vader_lexicon","punkt"])

from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()


[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package state_union to /root/nltk_data...
[nltk_data]   Unzipping corpora/state_union.zip.
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
"""
Sentiment Analysis for all the posts in the network.
"""
def sentimentScore(sentence):
  x=sia.polarity_scores(sentence)
  diff=abs(x['pos']-x['neg'])
  return 0 if diff<0.01 else 1 if x['pos']>x['neg'] else -1

In [ ]:
print(sentimentScore("Wow, NLTK is !"))
print(sentimentScore("Wow, I hate love hate NLTK!"))

1
1


In [ ]:
i=0
rows=[]
for node in G:
  listOfdata=(G.nodes[node]["data"]).split('{\'')[1:]
  print(i,"    ",G.nodes[node]["data"])
  pr=pageRank[str(node)]
  dc=degreeCentrality[str(node)]
  for l in listOfdata:
    print(l)

    data=(l).split('PostId\':')
    id1=(data[1].split('\'Title\':'))
    title1=(id1[1].split('\'Score\':'))
    score1=(title1[1].split('\'Date\':'))
    print(score1)
    date1=(score1[1].split('\'Comments\':'))
    postID = id1[0][2:-3]
    title = title1[0][2:-3]
    score = score1[0][2:-3]
    date = date1[0][2:-3]
    comments = date1[1][2:-3]
    # Below is all the data we require for a given post
    sA=sentimentScore(title)
    row=[postID,title,date,comments,sA,pr,dc,score]
    rows.append(row)
    print("X-Features---->",postID,"   ",title,"   ",date,"   ",comments,"   ",sA,"   ",pr,"   ",dc)
    print("Y-Features---->",score)
  i=i+1
  # if i>100:
  #   break
print("Data for ",(i-1)," posts has been converted to feature vector.")

In [ ]:
filename = "feature_vector.csv"
	
with open(filename, 'w') as csvfile:
	csvwriter = csv.writer(csvfile)
	csvwriter.writerows(rows)